## Things to be completed:
- What is $g(x,y)$ set to and why?

# Pressure waves in a box

In this lesson we examine the 2D wave equation simulating a pressure wave traversing through a rectangular box. This simulation relates closely to what occurs inside of a piezoelectrically driven printhead. In such devices a piezoelectric transducer periodically causes mechanical deformation of an ink channel, inputting pressure waves into to fluid which reflect around and intefere with eachother resulting in the despensing of droplets. See a simplified schematic below.

<img src="inkchannel.png">
<br/>
#### Figure 1. Piezoelecric inkjet printhead schematic

The dependent vaiable will be the pressure field but for simplicity we will derive all of the necessary equations with the dependent variable $u$.
<br/>

### The 2D wave Equation

The wave equation is a hyperbolic partial differential equation. In 2D it is written as follows:

\begin{equation} \label{eq:1}
\frac{\partial^2 u}{\partial t^2} = c^2 \nabla^2 u
\end{equation}

Or writing out the Laplacian, $\nabla^2$

\begin{equation} \label{eq:2}
\frac{\partial^2 u}{\partial t^2} = c^2\left(\frac{\partial^2 u}{\partial x^2} + \
\frac{\partial^2 u}{\partial y^2}\right)
\end{equation}

### Discretization
Here we are going to solve it using a central time, central space leapfrog method. First we need to descretize the equation. We will denote time steps by the letter $n$, spatial steps in the x and y directions will be denoted by $i$ and $j$ respectively. Applying the central difference in time and space the the left and righ hand sides gives:

\begin{equation} \label{eq:3}
\frac{u_{ij}^{n+1} - 2u_{ij}^{n} + u_{ij}^{n-1}}{\Delta t^{2}} =
c^2\left(\frac{u^{n}_{i+1,j} - 2u^{n}_{ij} + u^{n}_{i-1,j}}{\Delta x^{2}} +\
\frac{u^{n}_{i,j+1} - 2u^{n}_{ij} + u^{n}_{i,j-1}}{\Delta y^{2}}\right)
\end{equation}

<br/>
We then need to solve for the unknown variable $u_{ij}^{n+1}$
<br/>

Letting $\sigma_x = \frac{\Delta t^2 c^2}{\Delta x^2}$ and $\sigma_y = \frac{\Delta t^2 c^2}{\Delta y^2}$ the expression for u at the next time step becomes:

\begin{equation} \label{eq:4}
u_{ij}^{n+1} = \sigma_x^2(u_{i+1,j}^n + u_{i-1,j}^n) + \sigma_y^2(u_{i,j+1}^n + \
u_{i,j-1}^n) + 2(1-\sigma_x - \sigma_y)u_{ij}^n - u_{ij}^n
\end{equation}

The stencil for this descretization can be seen in Figure 2 below. We can see that the next time step at $n+1$ is dependent on information from the previous two time steps. The requires us to solve the first time at $n = 1$ seperately from the others. We will address this at the end of the next section once we have defined our initial conditions.

<img src="stencil.png">
#### Figure 2. Stencil for leapfrog scheme

### Initial conditions
Since we have a second order time derivative we need two initial conditions which will come in the form of $u(x,y,0) = f(x,y)$ and $\frac{\partial u}{\partial t} = g(x,y)$

Our initial pressure field will be a Gaussian function centered on the top edge of our rectangular domain and spread out along its width.

\begin{equation} \label{eq:5}
f(x,y) = A e^{-\left(\frac{(x - x_0)^2}{2\lambda_x^2} + \frac{(y - y_0)^2}{2\lambda_y^2}\right)}
\end{equation}

Where $x_0$ and $y_0$ are the center of the peak of the wave and $\lambda_x$ and $\lambda_y$ are a factor of "spread-out-ness" in the x and y directions respectively.

We set $g(x,y) = something$ becasue... We then discretize this to arrive at the expression for the first time step $u_{ij}^1$:

\begin{equation} \label{eq:6}
\frac{\partial u}{\partial t} = g(x,y) \rightarrow \frac{u_{ij}^{1} - u_{ij}^{-1}}{2\Delta t} \rightarrow u_{ij}^{-1} = u_{ij}^1 - 2\Delta t g(x,y)
\end{equation}

\begin{equation} \label{eq:7}
u_{ij}^1 = \frac{\sigma_x^2}{2}(u_{i+1,j}^0 + u_{i-1,j}^0) + \frac{\sigma_y^2}{2}(u_{i,j+1}^0 + u_{i,j-1}^0) + 2\Delta t g_{ij}
\end{equation}



### Boundary Conditions 
We want the wave to be reflected back into the domain at the boundaries. This can be done with either Dirichlet or Neumann boundary conditions. Explained in more detail [here](http://hplgit.github.io/INF5620/doc/pub/sphinx-wave/._main_wave003.html
).

We will be applying the Neumann boundary conditions so to preserve the sign of the reflected wave. The boundary conditions are defined using the normal derivitave of u at the boundaries:

\begin{equation} \label{eq:8}
\frac{\partial n}{\partial n}\equiv \textbf{n} \cdot\nabla u = 0
\end{equation}

The derivative with respect to n is in the outward normal direction from the boundary. 

To incorporate this condition into our scheme we apply the same sentral difference approximation. At the left hand vertical boundary where $i = 0$ we have

\begin{equation} \label{eq:9}
\frac{u_{-1,j}^n - u_{1,j}^n}{2\Delta x} = 0
\end{equation}

however, the value $u_{-1,j}$ is not defined in our domain. We can either make use of an extra layer of points all around our domain (ghost points), or by looking at equation $\ref{eq:5}$ and $\ref{eq:6}$ with the normal derivative set equal to zero, it can be seen that $u_{-1,j}^n = u_{1,j}^n$. We can then write the wave equation for the interior boundary points at $i=0$ as

\begin{equation} \label{eq:10}
u_{0,j}^{n+1} = 2\sigma_x^2 u_{1j}^n + 2\sigma_y^2(u_{0,j+1}^n + u_{0,j-1}^n)+\ 2(1 - \sigma_x^2 - \sigma_y^2)u_{0j}^n - u_{0j}^{n-1}
\end{equation}

Following the same rational, the equations for the other boundaries, including the corners are as follows:

Right boundary where $i = N_x$:
\begin{equation} \label{eq:11}
u_{N_x,j}^{n+1} = 2\sigma_x^2 u_{N_{x-1},j}^n + 2\sigma_y^2(u_{N_x,j+1}^n + u_{N_x,j-1}^n)+\ 2(1 - \sigma_x^2 - \sigma_y^2)u_{N_x,j}^n - u_{N_x,j}^{n-1}
\end{equation}

Bottom boundary where $j = 0$:
\begin{equation} \label{eq:12}
u_{i,0}^{n+1} = 2\sigma_x^2 (u_{i+1,0}^n + u_{i-1,0}) + 2\sigma_y^2u_{i,1}^n+\ 2(1 - \sigma_x^2 - \sigma_y^2)u_{i,0}^n - u_{i,0}^{n-1}
\end{equation}

Top boundary where $j = N_y$:
\begin{equation} \label{eq:13}
u_{i,N_y}^{n+1} = 2\sigma_x^2 (u_{i+1,N_y}^n + u_{i-1,N_y}) + 2\sigma_y^2u_{i,N_{y-1}}^n + 2(1 - \sigma_x^2 - \sigma_y^2)u_{i,N_y}^n - u_{i,N_y}^{n-1}
\end{equation}

Bottom left corner where $i = j = 0$:
\begin{equation} \label{eq:14}
u_{0,0}^{n+1} = 2\sigma_x^2 u_{1,0}^n + 2\sigma_y^2u_{0,1}^n + 2(1 - \sigma_x^2 - \sigma_y^2)u_{0,0}^n - u_{0,0}^{n-1}
\end{equation}

Top left corner where $i = 0, j = N_y$:
\begin{equation} \label{eq:15}
u_{0,N_y}^{n+1} = 2\sigma_x^2 u_{1,N_y}^n + 2\sigma_y^2u_{0,N_{y-1}}^n + 2(1 - \sigma_x^2 - \sigma_y^2)u_{0,N_y}^n - u_{0,N_y}^{n-1}
\end{equation}

Bottom right corner where $i = N_x, j = 0$:
\begin{equation} \label{eq:16}
u_{N_x,0}^{n+1} = 2\sigma_x^2 u_{N_{x-1},0}^n + 2\sigma_y^2u_{N_x,1}^n + 2(1 - \sigma_x^2 - \sigma_y^2)u_{N_x,0}^n - u_{N_x,0}^{n-1}
\end{equation}

Top right corner where $i = N_x, j = N_y$:
\begin{equation} \label{eq:17}
u_{N_x,N_y}^{n+1} = 2\sigma_x^2 u_{N_{x-1},N_y}^n + 2\sigma_y^2u_{N_x,N_{y-1}}^n + 2(1 - \sigma_x^2 - \sigma_y^2)u_{N_x,N_y}^n - u_{N_x,N_y}^{n-1}
\end{equation}


### Stability
The von Neumann stability condition for the 2D wave equation is the following:

\begin{equation} \label{eq:18}
\left(\frac{\Delta t}{\Delta x}\right) + \left(\frac{\Delta t}{\Delta y}\right) \leq 1 \rightarrow \Delta t = \left(\frac{\Delta x^2 \Delta y^2}{\Delta x^2 + \Delta y^2}\right)^{\frac{1}{2}}
\end{equation}

Details can be found [here](http://math.mit.edu/classes/18.086/2006/am53.pdf), expertly explained by Gilbert Strang.

### Python Implementation
We now create two functions, one for creating the initial conditions of the problem and another for calculating all the other time steps and applying boundary conditions.

In [1]:
from IPython.core.display import HTML
css_file = 'numericalmoocstyle.css'
HTML(open(css_file, "r").read())